In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.2
3.3


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x0000011627633380>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [01:52<00:00,  1.78it/s]
[MainProcess/INFO] experiments finished


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,120,228.023392,10.0,0.473518,198.679481,10.0,0.341862,320.614430,1.5,0.647263,...,4,9,3,0,4,1,2,4,0,dikesnet
1,18,260.541356,1.5,0.137455,116.847660,10.0,0.945624,290.547536,10.0,0.490350,...,4,9,3,0,4,1,2,5,0,dikesnet
2,45,212.172905,1.0,0.712211,78.508779,1.0,0.598337,197.653297,1.0,0.384091,...,4,9,3,0,4,1,2,6,0,dikesnet
3,110,123.090595,10.0,0.173836,101.096923,1.5,0.860186,307.395540,1.0,0.150357,...,4,9,3,0,4,1,2,7,0,dikesnet
4,128,42.362957,10.0,0.334475,309.604894,1.5,0.036029,140.536071,10.0,0.317008,...,4,9,3,0,4,1,2,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,101,321.847064,1.5,0.156866,57.765547,1.5,0.393779,117.699276,1.0,0.185093,...,9,7,0,7,0,9,4,49,3,dikesnet
196,86,193.009683,10.0,0.272478,302.197037,1.0,0.277668,60.039083,1.5,0.376331,...,9,7,0,7,0,9,4,50,3,dikesnet
197,7,32.041668,1.5,0.976560,257.084032,10.0,0.441433,265.998579,1.0,0.136225,...,9,7,0,7,0,9,4,51,3,dikesnet
198,130,221.299798,10.0,0.757452,155.305429,10.0,0.425393,173.129276,10.0,0.249017,...,9,7,0,7,0,9,4,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.266307e+08,0.0,1.135536e+08,0.000000,9.177486e+07,0.000000,4.663862e+07,0.000208,6.201594e+07,0.000000,1.287600e+09,182.340043
1,2.266307e+08,0.0,1.135536e+08,0.000000,9.177486e+07,0.000000,4.353304e+07,0.000000,6.201594e+07,0.000000,1.287600e+09,0.000000
2,2.266307e+08,0.0,1.135536e+08,0.000000,9.451915e+07,0.001090,4.353304e+07,0.000000,7.125212e+07,0.001873,1.287600e+09,850.110129
3,2.266307e+08,0.0,1.135536e+08,0.000000,1.174542e+08,0.006486,4.353304e+07,0.000000,6.467093e+07,0.000346,1.287600e+09,1070.647957
4,2.266307e+08,0.0,1.421604e+08,0.005697,9.177486e+07,0.000000,4.353304e+07,0.000000,6.201594e+07,0.000000,1.287600e+09,1911.476242
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.310212e+08,0.0,2.079608e+08,0.000000,1.149952e+08,0.000000,3.744155e+07,0.000000,1.113913e+08,0.000000,1.176700e+09,0.000000
196,2.310212e+08,0.0,2.079608e+08,0.000000,1.149952e+08,0.000000,3.744155e+07,0.000000,1.113913e+08,0.000000,1.176700e+09,0.000000
197,2.310212e+08,0.0,2.079608e+08,0.000000,1.149952e+08,0.000000,3.744155e+07,0.000000,1.113913e+08,0.000000,1.176700e+09,0.000000
198,2.310212e+08,0.0,2.079608e+08,0.000000,1.149952e+08,0.000000,3.744155e+07,0.000000,1.113913e+08,0.000000,1.176700e+09,0.000000


In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [49]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [02:51<00:00,  1.75it/s]
[MainProcess/INFO] experiments finished


In [50]:
experiments, outcomes = results

In [51]:
# only works because we have scalar outcomes
outcomes = pd.DataFrame(outcomes)
outcomes

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,3.298039e+07,0.039730,1.289850e+08,0.289399,0.000000e+00,0.000000,1.871795e+07,0.021106,253800000.0,0.0
1,5.397251e+07,0.000000,7.145393e+07,0.079595,1.813101e+08,0.379835,2.020112e+07,0.011057,0.000000e+00,0.000000,253800000.0,0.0
2,5.397251e+07,0.000000,4.324815e+07,0.040360,1.077573e+09,1.800792,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,5.167709e+07,0.040158,2.027031e+08,0.293885,6.688337e+07,0.025164,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,3.857094e+08,0.360337,1.295569e+08,0.233397,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2.002437e+09,1.317410,0.000000e+00,0.000000,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
296,0.000000e+00,0.000000,3.852942e+08,0.352901,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,1.306158e+08,0.114954,0.000000e+00,0.000000,2.879840e+07,0.000000,4.320964e+06,0.003148,0.000000e+00,0.000000,369700000.0,0.0
298,0.000000e+00,0.000000,2.066891e+07,0.020926,2.879840e+07,0.000000,4.171274e+07,0.020330,0.000000e+00,0.000000,369700000.0,0.0


In [52]:
# Calculate the 10th percentile
threshold = np.percentile(outcomes['A.5 Total Costs'], 50)
print(f"The threshold for 10% worst outcome for Total Costs equals {threshold}")

The threshold for 10% worst outcome for Total Costs equals 25497846.8713129


In [53]:
# Get the column to optimise
opt = outcomes['A.5 Total Costs']
# To dataframe
opt = pd.DataFrame(opt)

In [54]:
opt

,A.5 Total Costs
0,1.871795e+07
1,0.000000e+00
2,0.000000e+00
3,0.000000e+00
4,0.000000e+00
...,...
295,0.000000e+00
296,0.000000e+00
297,0.000000e+00
298,0.000000e+00


In [58]:
opt['A.5 Total Costs'] = np.where(opt['A.5 Total Costs'] < threshold, 1, 0)
y = opt

# y = opt['A.5 Total Costs'].to_numpy()

In [59]:
experiments = experiments.iloc[:,0:49]
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1
0,130,55.635437,10.0,0.536260,347.897301,10.0,0.598116,177.573116,1.0,0.469739,...,0,0,0,0,0,0,0,0,0,0
1,49,125.217433,10.0,0.619258,257.290345,10.0,0.385083,348.764327,1.5,0.438235,...,0,0,0,0,0,0,0,0,0,0
2,63,89.339671,10.0,0.733243,83.720656,1.0,0.584530,101.016752,1.5,0.257195,...,0,0,0,0,0,0,0,0,0,0
3,46,339.478461,1.0,0.364224,267.923682,1.0,0.657141,35.575916,1.0,0.507996,...,0,0,0,0,0,0,0,0,0,0
4,60,212.090699,1.5,0.074846,324.271409,1.5,0.188651,192.840893,1.5,0.360014,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,65,336.885328,1.5,0.009027,80.116372,1.5,0.249932,258.498316,10.0,0.300070,...,0,0,5,0,0,0,0,0,0,0
296,95,113.466513,1.0,0.687767,319.831050,1.5,0.101167,88.830880,10.0,0.788113,...,0,0,5,0,0,0,0,0,0,0
297,19,31.316571,1.5,0.255296,66.500620,10.0,0.922866,149.205489,1.0,0.967519,...,0,0,5,0,0,0,0,0,0,0
298,5,97.958703,1.0,0.747012,137.695750,1.0,0.550631,51.218877,1.0,0.454005,...,0,0,5,0,0,0,0,0,0,0


In [60]:
# PRIM: Patient Rule Induction Algorithm  (Open exploration scenario discovery)

from ema_workbench.analysis import prim
from ema_workbench import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Define x and y:
x = experiments

prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1)
box1 = prim_alg.find_box()

AssertionError: 

In [85]:
# Look at is the trade-off between coverage and density:

# Enable matplotlib interactive mode
# %matplotlib notebook

import matplotlib.pyplot as plt
 
box1.show_tradeoff(annotated=True)
plt.show()

NameError: name 'box1' is not defined